In [0]:
import numpy as np
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving data14smaller_3_2.csv to data14smaller_3_2.csv


In [0]:
!unzip "data16smaller_1.csv.zip"

Archive:  data18smaller_1.csv.zip
  inflating: data18smaller_1.csv     


In [0]:
data16smaller = pd.read_csv("data14smaller_3_2.csv", low_memory = False, dtype = {'subsidies_flag':"int64"})
data16smaller.head()

In [0]:
!pip install pgmpy==0.1.7
from pgmpy.estimators import HillClimbSearch, BicScore, BayesianEstimator
from pgmpy.models import BayesianModel
from sklearn.model_selection import train_test_split
from pgmpy.inference import VariableElimination

     |████████████████████████████████| 225kB 2.9MB/s 
     |████████████████████████████████| 1.3MB 43.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/27/d5/45/f9bd7e0f7d3752b706163ce2a0333d29855828c3e57a003640
Successfully built pgmpy
  Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2


In [0]:
from collections import defaultdict
from pgmpy.inference import VariableElimination
def temp_predict(model,data):
    missing_variables = list(set(model.nodes()) - set(data.columns))
    pred_values = defaultdict(list)
    model_inference = VariableElimination(model)
    for index, data_point in data.iterrows():
        states_dict = model_inference.map_query(variables=missing_variables, evidence=data_point.astype(int).to_dict())
        for k, v in states_dict.items():
            pred_values[k].append(v)
    return pd.DataFrame(pred_values, index=data.index)

In [0]:
data16train, data16test = train_test_split(data16smaller,random_state=9,test_size=0.2)
hc = HillClimbSearch(data16train, scoring_method=BicScore(data16train))
best_model = hc.estimate()
print(best_model.edges())

[('bldgclass', 'unit_ratio'), ('bldgclass', 'violations'), ('bldgclass', 'yearbuilt'), ('bldgclass', 'subsidies_flag'), ('residfar', 'subsidies_flag'), ('residfar', 'landuse'), ('residfar', 'yearbuilt'), ('residfar', 'bldgclass'), ('yearbuilt', 'unit_ratio'), ('yearbuilt', 'violations'), ('landuse', 'bldgclass')]


In [0]:
best_model.fit(data16train, estimator=BayesianEstimator, prior_type='K2')

for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    
print best_model.local_independencies('violations')

for thenode in best_model.nodes():
    print best_model.active_trail_nodes(thenode)

CPD of bldgclass:
+--------------+-----------------------+-----------------------+----------------------+-------------+---------------------+------------------------+------------------------+----------------------+-----------------------+----------------------+-----------------------+-----------------------+----------------------+-----------------------+-----------------------+-----------------------+-----------------------+----------------------+-----------------------+-----------------------+
| landuse      | landuse(0)            | landuse(0)            | landuse(0)           | landuse(0)  | landuse(0)          | landuse(1)             | landuse(1)             | landuse(1)           | landuse(1)            | landuse(1)           | landuse(2)            | landuse(2)            | landuse(2)           | landuse(2)            | landuse(2)            | landuse(3)            | landuse(3)            | landuse(3)           | landuse(3)            | landuse(3)            |
+--------------+--

In [0]:
best_model.fit(data16train, estimator=BayesianEstimator, prior_type='K2')
test_x = data16test.drop(columns=['violations'])
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(data16test.loc[:,'violations']==predicted_test['violations']).mean()

Out of sample: 0.6414532459797498


In [0]:
train_x = data16train.drop(columns=['violations'])
predicted_train = temp_predict(best_model, train_x)
print "In of sample:",(data16train.loc[:,'violations']==predicted_train['violations']).mean()

In of sample: 0.6534137443228353


In [0]:
data16smaller = pd.read_csv("data14smaller_4_2.csv", low_memory = False, dtype = {'subsidies_flag':"int64"})

In [0]:
data16train, data16test = train_test_split(data16smaller,random_state=9,test_size=0.2)
hc = HillClimbSearch(data16train, scoring_method=BicScore(data16train))
best_model = hc.estimate()
print(best_model.edges())

best_model.fit(data16train, estimator=BayesianEstimator, prior_type='K2')

train_x = data16train.drop(columns=['vio_rt'])
predicted_train = temp_predict(best_model, train_x)
print "In of sample:",(data16train.loc[:,'vio_rt']==predicted_train['vio_rt']).mean()

test_x = data16test.drop(columns=['vio_rt'])
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(data16test.loc[:,'vio_rt']==predicted_test['vio_rt']).mean()

[('bldgclass', 'unit_ratio'), ('bldgclass', 'subsidies_flag'), ('bldgclass', 'yearbuilt'), ('bldgclass', 'vio_rt'), ('residfar', 'subsidies_flag'), ('residfar', 'landuse'), ('residfar', 'yearbuilt'), ('residfar', 'bldgclass'), ('yearbuilt', 'unit_ratio'), ('yearbuilt', 'vio_rt'), ('landuse', 'bldgclass')]
In of sample: 0.37175191720646267
Out of sample: 0.38862418106015484


In [0]:
data16train, data16test = train_test_split(data16smaller,random_state=9,test_size=0.2)
hc = HillClimbSearch(data16train, scoring_method=BicScore(data16train))
best_model = hc.estimate()
print(best_model.edges())

best_model.fit(data16train, estimator=BayesianEstimator, prior_type='K2')

for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    
print best_model.local_independencies('vio_rt')

for thenode in best_model.nodes():
    print best_model.active_trail_nodes(thenode)

[('bldgclass', 'unit_ratio'), ('bldgclass', 'subsidies_flag'), ('bldgclass', 'yearbuilt'), ('bldgclass', 'vio_rt'), ('residfar', 'subsidies_flag'), ('residfar', 'landuse'), ('residfar', 'yearbuilt'), ('residfar', 'bldgclass'), ('yearbuilt', 'unit_ratio'), ('yearbuilt', 'vio_rt'), ('landuse', 'bldgclass')]
CPD of bldgclass:
+--------------+-----------------------+-----------------------+----------------------+-------------+---------------------+------------------------+------------------------+----------------------+-----------------------+----------------------+-----------------------+-----------------------+----------------------+-----------------------+-----------------------+-----------------------+-----------------------+----------------------+-----------------------+-----------------------+
| landuse      | landuse(0)            | landuse(0)            | landuse(0)           | landuse(0)  | landuse(0)          | landuse(1)             | landuse(1)             | landuse(1)         